In [4]:
import numpy as np
import h5py
from scipy.io import loadmat
import pandas as pd
from tqdm.notebook import tqdm
import os
import glob

In [ ]:
ALL_TEST_GRIDS = {}

In [ ]:
def load_test_data_perparticipant(participant_num):
    session_files = glob.glob(f"/Users/mishaal/personalproj/Generative-Replay/MEG_Behav/s{participant_num}/T*.mat")
    num_sessions = len(session_files)
    meg_data = loadmat(f"/Users/mishaal/personalproj/Generative-Replay/MEG_data/s{participant_num}/Data_inference.mat")
    classifier_data = loadmat(f"/Users/mishaal/personalproj/Generative-Replay/MEG_data/s{participant_num}/Class_data.mat")
    
    meg_signal_data = meg_data["data"]
    meg_correct_dup = meg_data["correct_trials_all"]
    bricks_conn_trial = meg_data["bricks_conn_trial"]
    bricks_rel_trial = meg_data["bricks_rel_trial"]
    stim_labels = meg_data["stimlabel"]# each unique presentation of a grid is given a label

    assert num_sessions * 48 == bricks_conn_trial.shape[0], "mismatch in trial numbers for participant %d" % participant_num

    #load the binomial classifiers 
    betas = classifier_data["betas_loc"]
    intercepts = classifier_data["intercepts_loc"]

    os.makedirs(f"/Users/mishaal/personalproj/clarion_replay/data/test_data/s{participant_num}", exist_ok=True)
    os.makedirs(f"/Users/mishaal/personalproj/clarion_replay/data/train_data/s{participant_num}", exist_ok=True)

    p_df = {"PID":[int(participant_num)]*bricks_conn_trial.shape[0], 
            "Session": [], "Trial": [], "Grid_Name": [],
            "left_element": [], "ontop_element": [], "right_element": [], "below_element": [],
            "besideness": [], "middle": [], "ontopness": [], 
              "Q_Brick_Middle": [], "Q_Brick_Left": [], "Q_Relation": [], "True Relation": [], "Correct": [], "RT":[]}

    absolute_trial_index = 0
    for idx, filename in enumerate(session_files):
        all_data = loadmat(filename)
        behav_data = all_data["res"][0, 0]["behav"][0,0]
        stimulus_grids = behav_data["SOLUTIONS_BUILT"]
        correctness = behav_data["correct"]
        rts = behav_data["rt"]
        q_stimuli = behav_data["stim_catch"]
        query_relation = behav_data["question_catch"] # a brick is presented in the middle and another on the top left corner. the relation of the top left brick to the middle brick is asked. This is the identity of the brick in the top left corner
        true_relation = behav_data["relation_catch"] # the relation in question
        
        # save experiment data
        n_trials = correctness.shape[0]
        p_df["Session"].extend([idx+1]*n_trials)
        p_df["Trial"].extend(list(range(1, n_trials+1)))
        stim_label = stim_labels[absolute_trial_index: absolute_trial_index + n_trials]
        p_df["Grid_Name"].extend(stim_label)

        p_df["left_element"].extend([bricks_rel_trial[i, 0] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])
        p_df["ontop_element"].extend([bricks_rel_trial[i, 1] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])
        p_df["right_element"].extend([bricks_rel_trial[i, 2] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])
        p_df["below_element"].extend([bricks_rel_trial[i, 3] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])

        p_df["besideness"].extend([bricks_conn_trial[i, 0] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])
        p_df["middle"].extend([bricks_conn_trial[i, 1] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])
        p_df["ontopness"].extend([bricks_conn_trial[i, 2] for i in range(absolute_trial_index, absolute_trial_index + n_trials)])

        p_df["Q_Brick_Left"].extend(q_stimuli[:, 0].flatten())
        p_df["Q_Brick_Middle"].extend(q_stimuli[:, 1].flatten())
        p_df["Q_Relation"].extend(query_relation)
        p_df["True Relation"].extend(true_relation)

        p_df["Correct"].extend(correctness.flatten())
        p_df["RT"].extend(rts.flatten())

        for j in range(n_trials): # add the grid
            if stim_label[absolute_trial_index + j] not in ALL_TEST_GRIDS:
                ALL_TEST_GRIDS[stim_label[absolute_trial_index + j]] = stimulus_grids[:, :, j]
            else:
                assert np.all(ALL_TEST_GRIDS[stim_label[absolute_trial_index + j]] == stimulus_grids[:, :, j]), f"Grids don't match participant {}, session {}, trial {}".format(participant_num, idx+1, j+1)

        absolute_trial_index += n_trials

    #Save MEG data
    np.save(f"/Users/mishaal/personalproj/clarion_replay/data/test_data/s{participant_num}/meg_data.npy", meg_signal_data)
    #save classifier data
    np.save(f"/Users/mishaal/personalproj/clarion_replay/data/test_data/s{participant_num}/classifier_data.npy", [betas, intercepts])    
    assert np.all(meg_correct_dup == np.ndarray(p_df["Correct"])), "correctness mismatch for participant %d" % participant_num 

    p_df = pd.DataFrame(p_df)
    p_df.to_csv(f"/Users/mishaal/personalproj/clarion_replay/data/test_data/s{participant_num}/test_data.csv")

    return p_df





SyntaxError: f-string: valid expression required before '}' (3970785379.py, line 62)

In [ ]:
def load_train_data_perparticipant(participant_num):
    pass

(6, 6, 48)